In [17]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
device = 'mps'

dataset_path = '/Users/soyoung/ml_project/study/train_dataset.pkl'

In [2]:
import pickle

with open(dataset_path, 'rb') as f:
  train_dataset = pickle.load(f)

In [3]:
# 데이터 준비
# 데이터 전처리
# train/test data
# dataloader
# model
# loss function
# optimizer
# hyper parameter
# training + test
# visualization

In [4]:
label_set = {'가구수정': 0,
 '걸레받이수정': 1,
 '곰팡이': 2,
 '꼬임': 3,
 '녹오염': 4,
 '들뜸': 5,
 '면불량': 6,
 '몰딩수정': 7,
 '반점': 8,
 '석고수정': 9,
 '오염': 10,
 '오타공': 11,
 '울음': 12,
 '이음부불량': 13,
 '창틀,문틀수정': 14,
 '터짐': 15,
 '틈새과다': 16,
 '피스': 17,
 '훼손': 18}

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(train_dataset, test_size=0.2)

train_data = [(x[0],label_set[x[1]]) for x in X_train]

In [7]:
import cv2
height = 400
width = 600
train_data = [(cv2.resize(x[0], (400,600)).reshape(3, height, width), label_set[x[1]]) for x in X_train]
test_data = [(cv2.resize(x[0], (400,600)).reshape(3, height, width), label_set[x[1]]) for x in X_test]
# train_data = [(x[0]/255., label_set[x[1]]) for x in X_train[:len(X_train)//2]]
# test_data = [(x[0]/255., label_set[x[1]]) for x in X_test[:len(X_test)//2]]

In [8]:
# hyper parameter
BATCH_SIZE = 64
EPOCHS = 100
LEARNING_RATE = 1e-3

In [9]:
train_loader = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size = BATCH_SIZE, shuffle=True)

In [10]:
from torchvision import models

# pre trained model
model = models.resnet18()

In [11]:
model.fc = nn.Linear(in_features=512, out_features=19)

In [12]:
from torch import optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [27]:
import numpy as np
from tqdm.auto import tqdm # 진행상황 나타내는 프로그레스 바 표시
import matplotlib.pyplot as plt
# device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
device = torch.device('mps')
total_acc = []
for epoch in range(EPOCHS):
  # 학습
  train_loss=[] # 학습시 나타나는 로스를 종합 -> 이후에 그래프 표시를 위함
  for img, label in train_loader:
    optimizer.zero_grad()
    pred = model((img/255.).to(device))
    loss = criterion(pred, label.to(device))
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())
  # 평가
  test_loss=[]
  test_sources = []
  with torch.no_grad():
    for img, label in test_loader:
      pred = model((img/255.).to(device))
      loss = criterion(pred, label.to(device))
      # pred = model(img)
      # loss = criterion(pred, label)
      test_loss.append(loss.item())
      test_sources.append((img[pred.cpu().argmax(axis=1)!=label.cpu()],
                           label[pred.cpu().argmax(axis=1)!=label.cpu()],pred.cpu().argmax(axis=1)[pred.cpu().argmax(axis=1)!=label.cpu()])) # label과 pred 값이 다른 부분만 test_sources에 담기
  # 노트북 파일 기존 프린트 되었던 화면을 지우는 코드
  clear_output()

  # 인식 잘 안된 부분의 시각화를 위해서 img, label, pred를 모아서 정리
  imgs = [x[0] for x in test_sources]
  imgs = torch.cat(imgs, axis=0)
  labels = [x[1] for x in test_sources]
  labels = torch.cat(labels, axis=0)
  preds = [x[2] for x in test_sources]
  preds = torch.cat(preds, axis=0)

  # 정확도 계산을 위한 코드. 다양하게 다른 방법도 가능함
  # sklearn의 metric 이용하는 방법 추천
  wrongs = [len(x[0]) for x in test_sources]
  acc = round(100-(sum(wrongs)/100),2)

  # 정확도 추이를 보기 위하여 각 에폭에서의 정확도 정보 수집
  total_acc.append(acc)

  # 학습 진행 상황 출력
  print("{}번째 train_loss : {} test_loss : {} Accuracy : {}%".format(epoch, round(np.array(train_loss).mean(),2),round(np.array(test_loss).mean(),2),
                                                                   acc))
  if img.shape[0]<1:continue # 혹시 더이상 오분류 내용이 없으면 지나가도록 세팅

  # 정확도 그래프 그리기
  plt.plot(total_acc)
  plt.show()

  # 오분류 내용이 어떤건지 알기 위해 오분류 내용 4개 뽑아서 label과 pred 글자 넣어서 표시
  for i in range(4):
    plt.subplot(1,4,i+1)    
    plt.imshow(imgs[i][0], cmap='gray')    
    plt.title("{} vs {}".format(labels[i].item(),preds[i].item()))
    plt.axis('off')
  plt.show()

RuntimeError: Input type (MPSFloatType) and weight type (torch.FloatTensor) should be the same